In [114]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(Market Beat or Market Meet)

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [175]:
# from Abbas

tickers_file_name = "Tickers_Example.csv"
tickers_file_original = pd.read_csv(tickers_file_name, header = None, names=['Ticker'])

total_investment = 1000000
start_date = "2023-10-01"
end_date = "2024-09-30"
drop_tickers = []

for tick in tickers_file_original['Ticker']:    

    try:

        ticker = yf.Ticker(tick) 

        # check if ticker is available
        if ticker.info.get('quoteType')=='NONE':
            # print("phewww", tick)
            tickers_file_original = tickers_file_original[tickers_file_original['Ticker'] != tick]
            continue

        # check if its in US or CAD market
        if ticker.fast_info.get('currency') not in ['USD', 'CAD']:
            # print("hmmm", tick)
            tickers_file_original = tickers_file_original[tickers_file_original['Ticker'] != tick]
            continue

        history = ticker.history(start = start_date, end = end_date)

        trading_days = history['Volume'].resample('ME').count()
        volume = history['Volume'].resample('ME').sum()
        trading_days = trading_days[trading_days>=18] # how does this work?
        volume = volume[volume.index==trading_days.index]
        monthly_volume = volume.mean()

        if monthly_volume<100000: 
            # print("sheesh", tick)
            tickers_file_original = tickers_file_original[tickers_file_original['Ticker'] != tick]

    except:
            # print("ufff", tick)
            pass
        
tickers_file_original = tickers_file_original.reset_index(drop=True)


In [203]:
# from Gurmehar

# Fetch historical data for both indices
sp500 = yf.Ticker("^GSPC").history(start=start_date, end=end_date)['Close']
tsx60 = yf.Ticker("XIU.TO").history(start=start_date, end=end_date)['Close']

sp500.index = pd.to_datetime(sp500.index)
tsx60.index = pd.to_datetime(tsx60.index)

# Ensure both dataframes align by date (in case they have missing values)
combined_data = pd.concat([sp500, tsx60], axis=1, keys=['S&P 500', 'TSX 60'])
combined_data = combined_data.dropna()  # Remove any rows with missing data

# Calculate the arithmetic mean for each day
combined_data['Arithmetic Mean'] = combined_data.mean(axis=1)

def get_stock_correlations(tickers, combined):
    correlations = {}
    
    # Download stock data for each ticker
    for ticker in tickers:
        stock_data = yf.Ticker(ticker).history(start=start_date, end=end_date)["Close"]
        
        
        # Calculate the correlation with the average index values
        correlation = combined["Arithmetic Mean"].corr(stock_data)
        correlations[ticker] = correlation
    
    return correlations

# Load your CSV file (make sure it has a column "Ticker" with stock tickers)
tickers_df = pd.read_csv("Tickers_Example.csv")
ticker_list = list(tickers_df.iloc[:,0])
correlations = get_stock_correlations(tickers_file_original.Ticker,combined_data)

# My own code: turn into datafram
beta_df = pd.DataFrame(correlations.items(), columns = ['Ticker', 'Beta'])

display(beta_df)

,Ticker,Beta
0,AAPL,0.677388
1,ABBV,0.853363
2,ABT,0.572666
3,ACN,0.081726
4,AIG,0.880958
5,AMZN,0.923711
6,AXP,0.970801
7,BA,-0.534248
8,BAC,0.964987
9,BB.TO,-0.865229


In [197]:
# separate by industry
#display(tickers_file_original)

# create dictionary to store tickers in each industry
industry_groups = {}

for i in range(len(tickers_file_original)):
    try: 
        name = tickers_file_original.loc[i, "Ticker"]
        stock = yf.Ticker(name)
        #print(stock)
        industry = stock.info['sector']
        #print(industry)
        if industry not in industry_groups:
            industry_groups[industry] = []
            
        industry_groups[industry].append(name)
    except:
        pass
#print(industry_groups)

# chat gpt 
max_length = max(len(stocks) for stocks in industry_groups.values())
df = pd.DataFrame({industry: stocks + ['-'] * (max_length - len(stocks))
                   for industry, stocks in industry_groups.items()})

# loop through each industry
#display(df)

In [209]:
# CATEGORY WEIGHTING
size_weight = 0.1
beta_weight = 0.5
in_sp500_weight = 0.05

# --RANKING BASED ON BETA--
beta_df['Distance to 1'] = abs(beta_df['Beta']-1)
beta_df['Score'] = (1 - beta_df['Distance to 1'])*100 # this could return negative scores

# this normalizes score to all between 0 and 100
#beta_df['Sorted'] =  beta_df["Distance to 1"].rank(ascending=True)
#max_rank = beta_df['Sorted'].max()
#beta_df['Score'] = (1 - (beta_df["Sorted"] - 1) / (max_rank - 1)) * 100

# -RANKING BASED ON COMPANY SIZE--
def get_market_cap(ticker):
    try:
        info = yf.Ticker(ticker).fast_info
        return info['marketCap']
    except:
        return 0

# --RANKING BASED ON WHETHER OR NOT COMPANY IS IN S&P500--

# Scrape the list of S&P 500 companies from Wikipedia
# next 3 lines from chat gpt
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_df = pd.read_html(url)[0]
sp500companies = sp500_df.Symbol.tolist()

def in_sp500(Ticker, sp500companies):
    if Ticker in sp500companies:
        return 100
    else:
        return 0

# --COMPILING ALL RANKINGS INTO ONE DATAFRAME--
rankings = tickers_file_original.copy()

rankings['Beta'] = beta_weight * beta_df['Score']
rankings["In S&P 500?"]= in_sp500_weight * rankings["Ticker"].apply(lambda x: in_sp500(x, sp500companies))

rankings["Market Cap"] = rankings["Ticker"].apply(get_market_cap)
rankings["Size Rank"] = rankings["Market Cap"].rank(ascending=False)
max_rank = rankings["Size Rank"].max()
rankings["Company Size"] = size_weight * round((1 - (rankings["Size Rank"] - 1) / (max_rank - 1)) * 100, 2)
rankings.drop(["Market Cap", "Size Rank"], axis=1, inplace=True)

rankings.set_index("Ticker", inplace=True)
rankings['Total Score'] = rankings.sum(axis=1)
display(rankings)

#print(sp500companies)


,Beta,In S&P 500?,Company Size,Total Score
Ticker,,,,
AAPL,33.869399,5.0,10.000,48.869399
ABBV,42.668134,5.0,8.333,56.001134
ABT,28.633315,5.0,6.389,40.022315
ACN,4.086283,5.0,7.222,16.308283
AIG,44.047880,5.0,0.833,49.880880
AMZN,46.185549,5.0,9.722,60.907549
AXP,48.540050,5.0,6.667,60.207050
BA,-26.712421,5.0,2.500,-19.212421
BAC,48.249344,5.0,8.611,61.860344


In [213]:
top_20_tickers = rankings.nlargest(20, 'Total Score')
print(top_20_tickers)

             Beta  In S&P 500?  Company Size  Total Score
Ticker                                                   
LLY     47.576421          5.0         9.444    62.020421
BAC     48.249344          5.0         8.611    61.860344
AMZN    46.185549          5.0         9.722    60.907549
AXP     48.540050          5.0         6.667    60.207050
PG      45.471971          5.0         8.889    59.360971
TXN     46.435139          5.0         5.556    56.991139
KO      43.544882          5.0         8.056    56.600882
ABBV    42.668134          5.0         8.333    56.001134
C       46.933725          5.0         3.333    55.266725
RY.TO   47.060734          0.0         7.500    54.560734
CAT     43.862301          5.0         5.278    54.140301
BK      47.793472          5.0         1.111    53.904472
CL      47.384731          5.0         1.389    53.773731
BLK     43.878425          5.0         4.722    53.600425
QCOM    43.351178          5.0         5.000    53.351178
AIG     44.047

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.